In [ ]:
#| default_exp model
%load_ext autoreload
%autoreload 2

In [ ]:
# | export
import os
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.base import ClassifierMixin, BaseEstimator
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import joblib
import yaml
import numpy as np
from pathlib import Path
from dataclasses import dataclass
from paragraph_detective.data_prep import prepare_data_from_csv, prepare_data_from_doc, create_line_features

In [ ]:
# | export
from typing import Final

MODEL_FILE_NAME: Final[str] = "clf.joblib"
X_COLS: Final[list[str]] = [
    "txt_len",
    "end_with_end_sent",
    "end_with_hyphen",
    "start_with_upper",
    "start_with_bullet",
    "diff_len_prev",
    "diff_max_len",
]


def df_to_x_y(train_df) -> tuple[np.ndarray, np.ndarray]:
    y_col = "new_paragraph"
    x = train_df[X_COLS]
    y = train_df[y_col]

    return x, y


def cross_validate_model(clf: BaseEstimator, train_df: pd.DataFrame, cv: int = 3) -> list[float]:
    x, y = df_to_x_y(train_df)
    return cross_val_score(clf, x, y, cv=cv).tolist()


@dataclass
class ModelScores:
    cv_scores: list[float]
    train_accuracy: float
    test_accuracy: float


def save_experiment(clf: BaseEstimator, scores: ModelScores, model_name: str, version: str):
    exp_dir: Path = Path("../experiments") / model_name / version
    os.makedirs(exp_dir, exist_ok=True)

    joblib.dump(clf, exp_dir / MODEL_FILE_NAME)
    metadata = dict(
        {
            "name": model_name,
            "version": version,
            "cv_scores": scores.cv_scores,
            "train_accuracy": scores.train_accuracy,
            "test_accuracy": scores.test_accuracy,
            "hparams": clf.get_params(),
        }
    )
    with open(exp_dir / "metadata.yaml", "w") as f:
        yaml.dump(metadata, f, sort_keys=False)


def save_model(clf: BaseEstimator):
    model_dir = Path("../model")
    os.makedirs(model_dir, exist_ok=True)
    joblib.dump(clf, model_dir / MODEL_FILE_NAME)


def train_model(
    clf: ClassifierMixin,
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    model_name: str,
    version: str,
    cv: int = 3,
) -> tuple[ClassifierMixin, ModelScores]:
    cv_scores = cross_validate_model(clf, train_df, cv=cv)
    x_train, y_train = df_to_x_y(train_df)
    clf.fit(x_train, y_train)
    y_pred_train = clf.predict(x_train)
    train_accuracy = float(accuracy_score(y_train, y_pred_train))
    x_test, y_test = df_to_x_y(test_df)
    y_pred_test = clf.predict(x_test)
    test_accuracy = float(accuracy_score(y_test, y_pred_test))
    scores = ModelScores(cv_scores=cv_scores, train_accuracy=train_accuracy, test_accuracy=test_accuracy)
    save_experiment(clf, scores, model_name, version)
    return clf, scores


In [ ]:
# manage experiments with diff models and then select the best one which is put into model.
train_df = prepare_data_from_csv(Path("../data/train.csv"))
test_df = prepare_data_from_csv("../data/test.csv")
clf, scores = train_model(GradientBoostingClassifier(), train_df, test_df, model_name="xg", version="1.0", cv=3)

# save_model(clf)
scores


In [ ]:
# | export
def load_model() -> ClassifierMixin:
    clf = joblib.load(Path("../model/clf.joblib"))
    return clf


In [ ]:
lclf = load_model()
lclf


In [ ]:
# | export
def clean_paragraph_from_txt_lines(clf: ClassifierMixin, lines: list[str]) -> str:
    lines_df = create_line_features(lines)
    x = lines_df[X_COLS]
    preds = clf.predict(x).tolist()
    txt = ""
    for l, pred in zip(lines, preds):
        if len(l) > 0:
            if l[-1] == "-":
                l = l[:-1]

        # 1 is for new paragraph.
        if pred == 1:
            txt += "\n" + l
        else:
            txt += l
    return txt

In [ ]:
with open(Path("../test_data/doc_a.txt"), "r") as f:
    lines: list[str] = f.read().split('\n')
    txt = clean_paragraph_from_txt_lines(clf, lines)
print(txt)

In [ ]:
import nbdev

nbdev.nbdev_export("core.ipynb")